<a href="https://colab.research.google.com/github/lephuocdat2000/AI-Tempo-Run/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
def Fixed_Coordinate(bbox):
  bbox = np.array(bbox)
  bbox = np.reshape(bbox,(4,2))
  xmin,xmax = np.amin(bbox[:,0],axis = 0),np.amax(bbox[:,0],axis=0)
  ymin,ymax = np.amin(bbox[:,1],axis=0), np.amax(bbox[:,1],axis=0)
  return xmin,ymin,xmax,ymax

def IOU_Calculation(gt_bbox,training_bbox):
  xmin_gt,ymin_gt,xmax_gt,ymax_gt = Fixed_Coordinate(gt_bbox)
  xmin,ymin,xmax,ymax = Fixed_Coordinate(training_bbox)
  if (xmax_gt < xmin) or (ymax_gt<ymin) or (xmin_gt > xmax) or (ymin_gt>ymax): return 0.0
  GT_bbox_area = (xmax_gt - xmin_gt + 1 ) * (ymax_gt - ymin_gt + 1 )
  Pre_bbox_area = (xmax - xmin + 1 ) * (ymax - ymin + 1 )
  xmin = max(xmin_gt,xmin)
  ymin = max(ymin_gt,ymin)
  xmax = min(xmax_gt,xmax)
  ymax = min(ymax_gt,ymax)
  intersection_area = (xmax-xmin+1)*(ymax-ymin+1)
  union_area = GT_bbox_area + Pre_bbox_area - intersection_area
  return intersection_area/union_area

In [3]:
cd /content/gdrive/MyDrive/AITempoRun

/content/gdrive/MyDrive/AITempoRun


In [4]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

Using CRAFT

In [ ]:
dictionary = os.listdir('result')

for file in dictionary:
    if file.endswith(".txt"):
       f1 = open(os.path.join('result',file),"r").readlines()
       if len(f1)!=0:
          image = cv2.imread(os.path.join('train/images',file[4:-4]+'.jpg'))
          f2 = open(os.path.join('train/labels',file[4:]),"r").readlines()
          f3 = open(os.path.join('Preprocessing Data/test',file[4:]),"w")
          mark = np.zeros((len(f2),))
          for i in range(len(f1)):
              coordinate1 = f1[i].split(",")
              coordinate1 = [int(i) for i in coordinate1]
              IoU_max, selection = 0.0,coordinate1
              for j in range(len(f2)):
                if mark[j]==0:
                    fence = f2[j].rfind(",")
                    coordinate2 = f2[j][0:fence].split(",")
                    coordinate2 = [int(i) for i in coordinate2]
                    IoU = IOU_Calculation(coordinate1,coordinate2)
                    if IoU >= 0.1: 
                        mark[j] = 1
                        if IoU_max<IoU : IoU_max,selection = IoU,coordinate2
              string = ""
              for idx,coor in enumerate(selection):
                    string = string + str(coor) 
                    if idx<len(selection)-1: string+=","
              f3.writelines(string+"\n")      
          # missing = np.where(mark==0)[0]
          # print(file,missing)
          # if len(missing!=0):
          #     for i in missing:
          #        fence = f2[i].rfind(",")
          #        miss_coordinate = f2[i][0:fence]
          #        f3.writelines(miss_coordinate+"\n")   
          f3.close()
          cv2.imwrite(os.path.join('Preprocessing Data/images',file[4:-4]+'.jpg'),image)                       

Average IOU

In [ ]:
label_dict = os.listdir('Preprocessing Data/Labeling with CRAFT')
IoU = 0
count = 0
for file in label_dict:
   f1 = open(os.path.join('result',"res_"+file),"r").readlines()
   f2 = open(os.path.join('Preprocessing Data/Labeling with CRAFT',file),"r").readlines()
   if len(f1)!=len(f2): 
     print(file)
     break
   for i in range(len(f1)):
        coordinate1 = f1[i].split(",")
        coordinate1 = [int(i) for i in coordinate1]
        coordinate2 = f2[i].strip().split(",")
        coordinate2 = [int(i) for i in coordinate2]
        IoU += IOU_Calculation(coordinate1,coordinate2)
        count+=1

print(IoU / count)

0.7936060866031666


Average Coordinate

In [ ]:
dictionary = os.listdir('Preprocessing Data/images')
cc = 0
for file in dictionary:
   f1 = open(os.path.join('train/labels',file[:-4]+'.txt'),"r").readlines()
   f2 = open(os.path.join('Preprocessing Data/Average Coordinate',file[:-4]+'.txt'),"w")
   cc+=1
   mask = np.zeros((len(f1),))
   for i in range(len(f1)):
        if mask[i]==0:
            fence = f1[i].rfind(",")
            coordinate1 = f1[i][0:fence].split(",")
            coordinate1 = [int(i) for i in coordinate1]
            average_coor = np.array(coordinate1)
            mask[i],count=1,1
            for j in range(i,len(f1)):
              if mask[j]==0:    
                fence = f1[j].rfind(",")
                coordinate2 = f1[j][0:fence].split(",")
                coordinate2 = [int(i) for i in coordinate2]
                IoU = IOU_Calculation(coordinate1,coordinate2)
                if IoU >= 0.1: 
                      mask[j]=1
                      average_coor += np.array(coordinate2)
                      count+=1
            average_coor = average_coor//count
            string = ""
            for idx,coor in enumerate(average_coor):
                    string = string + str(coor) 
                    if idx<len(average_coor)-1: string+=","
            f2.writelines(string+"\n") 
   f2.close()   

print(cc)

Inverse

In [6]:
import itertools 

In [36]:
dictionary = os.listdir('train/labels')

for file in dictionary:
    print(file)
    dicts = {}
    f1 = open(os.path.join('result',"res_"+file),"r").readlines()
    if len(f1)!=0:
        f2 = open(os.path.join('train/labels',file),"r").readlines()
        f3 = open(os.path.join('Preprocessing Data/Inverse',file),"w")
        #create IoU matrix and coordinate dictionary
        if len(f2)>1: 
          list_indexes = list(itertools.combinations(range(len(f2)),2))
          IoU_train_label = np.zeros((len(f2),len(f2)))
          for i,pair in enumerate(itertools.combinations(f2,r=2)):
              index_1 = int(list_indexes[i][0])
              index_2 = int(list_indexes[i][1])
              if index_1 in dicts:  coordinate1 = dicts[index_1]
              else: 
                  coordinate1 = f2[index_1]
                  fence = coordinate1.rfind(",")
                  coordinate1 = coordinate1[0:fence]
                  coordinate1 = [int(i) for i in coordinate1.split(",")]
                  dicts[index_1] = coordinate1 
              if index_2 in dicts: coordinate2 = dicts[index_2]
              else: 
                  coordinate2 = f2[index_2]
                  fence = coordinate2.rfind(",")
                  coordinate2 = coordinate2[0:fence]
                  coordinate2 = [int(i) for i in coordinate2.split(",")]
                  dicts[index_2] = coordinate2 
              IoU_train_label[index_1][index_2] = IOU_Calculation(coordinate1,coordinate2)  
          
          mask = np.zeros((len(f2),))
          for i in range(len(f2)):
            if mask[i]==0:
              overlaps = np.where(IoU_train_label[i,:]>0.2)[0]
              max_ , selection,text = 0.0,dicts[i],f2[i][f2[i].rfind(",")+1:].strip()
              for j in range(len(f1)):
                  coordinate1 = dicts[i]
                  coordinate2 =[int(i) for i in f1[j].split(",")]
                  IoU = IOU_Calculation(coordinate1,coordinate2)
                  if IoU >= 0.1:
                    max_,mask[i] = IoU,1 
                    for idx in overlaps:
                        IoU_ = IOU_Calculation(dicts[idx],coordinate2)
                        mask[idx] = 1
                        if IoU_ > max_: max_,selection,text= IoU_, dicts[idx],f2[idx][f2[idx].rfind(",")+1:].strip()

              if max_==0.0:
                  total = np.array(selection)
                  for idx in overlaps:
                     total += np.array(dicts[idx])
                     mask[idx] = 1
                  average = (total // (len(overlaps)+1)).astype('int')
                  max_= IOU_Calculation(selection,average)
                  for idx in overlaps:
                     IoU = IOU_Calculation(dicts[idx],average)
                     if IoU > max_: 
                       max_ = IoU
                       selection = dicts[idx]   
                       text = f2[idx][f2[idx].rfind(",")+1:].strip()
            
              string = ""
              for idx,coor in enumerate(selection):
                    string = string + str(coor) + ","
              string += text
              print(string)
              f3.writelines(string+'\n')
          f3.close()    
        else: 
         fence = f2[0].rfind(",")
         coordinate = f2[0][0:fence].split(",")
         text = f2[0][fence+1:].strip()
         coordinate = [int(i) for i in coordinate]
         string = ""
         for idx,coor in enumerate(coordinate):
             string = string + str(coor) + ","
         string+=text
         f3.writelines(string+'\n')
         f3.close()    
    #break
           

Streaming output truncated to the last 5000 lines.
240,207,320,195,318,232,243,238,Café
351,349,370,350,374,369,353,371,HẠNH
378,351,398,352,399,368,379,366,LHVL
435,357,478,358,480,374,435,371,0703.949.616
242,144,261,147,266,172,241,171,CÀ
386,164,432,169,435,207,381,199,TẤN
267,153,304,157,304,178,272,176,PHÊ
421,353,433,356,435,371,421,372,ĐT:
297,150,372,162,378,198,306,187,VĨNH
302,347,325,346,324,364,298,364,HƯNG
240,345,259,348,260,366,243,364,ĐC:
261,346,277,348,280,366,264,365,1/13
330,347,348,345,349,366,330,367,TÂN
229,241,305,244,307,336,243,340,NỮA
387,253,472,267,474,344,395,340,TRĂNG
304,226,387,235,391,345,316,342,VẦNG
train_1218.txt
336,239,365,235,363,285,332,285,ĐỀ
281,92,334,91,329,104,278,104,THÔNG
239,186,263,187,266,225,242,223,RA
192,92,234,90,234,102,193,103,TOÀN
416,92,448,92,445,104,415,104,THƯ
391,90,413,90,412,104,391,106,VŨ
179,243,196,242,193,285,176,282,Ý
278,241,326,243,326,285,275,286,SÁT
284,124,395,125,400,160,285,160,ĐƯỜNG
122,241,172,239,171,282,1